In [1]:
from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F

/home/z40362k/anaconda3/envs/action/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_model_name_or_path="openai/clip-vit-base-patch16"
text_model = CLIPModel.from_pretrained(text_model_name_or_path)
txt_tokenizer = CLIPProcessor.from_pretrained(text_model_name_or_path)

In [3]:
labels = ["BasketballDunk", "Billiards", "CleanAndJerk", "CliffDiving", "CricketBowling", "CricketShot", "Diving", "FrisbeeCatch", "GolfSwing", "HighJump", "JavelinThrow", "LongJump", "Shotput", "TennisSwing", "VolleyballSpiking", "BaseballPitch", "HammerThrow", "PoleVault", "SoccerPenalty", "ThrowDiscus"]

In [4]:
txt_labels = txt_tokenizer(labels, padding=True, return_tensors="pt")
txt_labels_feats = text_model.get_text_features(**txt_labels)

In [5]:
txt_labels_feats.shape

torch.Size([20, 512])

In [6]:
# I want to save the cosine similarity between each label
save = []

for i in range(txt_labels_feats.shape[0]):
    for j in range(txt_labels_feats.shape[0]):
        if i == j:
            continue

        txt_consine = F.cosine_similarity(txt_labels_feats[i].unsqueeze(0), txt_labels_feats[j].unsqueeze(0), dim=1)
        save.append([labels[i], labels[j], txt_consine.item()])
                            


In [7]:
# sort save by cosine similarity
save.sort(key=lambda x: x[2], reverse=True)

# Remove 0,2,4,6... because they are the same label
save = save[1::2]


In [8]:
pick_item = []
score = []
un_pick = []

for item in save:

    if item[0] in un_pick:
        continue
    un_pick.append(item[1])
    un_pick.append(item[0])
    pick_item.append(item[0])
    score.append(item[2])

In [9]:
for i in range(len(pick_item)):
    print(pick_item[i])

LongJump
HammerThrow
ThrowDiscus
TennisSwing
CricketShot
Diving
VolleyballSpiking
SoccerPenalty
PoleVault
BaseballPitch
CleanAndJerk
Billiards
FrisbeeCatch
JavelinThrow


In [45]:
score

[0.9187135696411133,
 0.9117065072059631,
 0.8978204727172852,
 0.8733699321746826,
 0.8629214763641357,
 0.8493157625198364,
 0.8414521813392639,
 0.8182151317596436,
 0.7342582941055298,
 0.6886559128761292]

In [ ]:
# TennisSwing
# LongJump
# CricketShot
# ThrowDiscus
# HammerThrow
# Diving
# SoccerPenalty
# GolfSwing

# GolfSwing
# HighJump
# CricketBowling
# HammerThrow
# Shotput
# CliffDiving
# VolleyballSpiking
# BasketballDunk